<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Chapter 2: 文本处理

Packages that are being used in this notebook:

In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0
tiktoken version: 0.9.0


- 本章涵盖数据准备和采样，以使输入数据"准备好"用于 LLM

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="800px">

## 2.1 理解词嵌入

- 本节不包含代码
- 嵌入向量有很多形式；在本书中我们专注于文本嵌入

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="800px">

- LLM 在高维空间中处理嵌入向量（即数千个维度）
- 由于我们无法可视化这种高维空间（我们人类思考通常是1、2或3维的），下图展示了一个二维嵌入空间

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="600px">

## 2.2 文本分词

- 在本节中，我们将对文本进行分词，这意味着将文本分解成更小的单元，例如单个单词和标点符号

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="600px">

- 加载我们想要处理的原始文本
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) 是一个公共领域的短篇故事

In [3]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

- （如果执行上一个代码单元时遇到 `ssl.SSLCertVerificationError`，可能是因为使用了过时的 Python 版本；你可以在 [GitHub 上找到更多信息](https://github.com/rasbt/LLMs-from-scratch/pull/403)）

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是对这段文本进行分词并嵌入，以供 LLM 使用
- 让我们基于一些简单的样本文本开发一个简单的分词器，然后可以将其应用于上面的文本
- 以下正则表达式将按空格进行分割

In [5]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 我们不仅想按空格分割，还想按逗号和句号分割，所以让我们修改正则表达式来实现这一点

In [6]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 正如我们所见，这会创建空字符串，让我们将它们移除

In [7]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 这看起来不错，但让我们也处理其他类型的标点符号，比如句号、问号等等

In [8]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 这已经很不错了，我们现在可以将这种分词方法应用到原始文本中。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="700px">

In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 让我们计算一下标记的总数。

In [10]:
print(len(preprocessed))

4690


## 2.3 将标记转换为标记 ID

- 接下来，我们将文本标记转换为标记 ID，以便稍后通过嵌入层进行处理。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="700px">

- 从这些标记中，我们现在可以构建一个包含所有唯一标记的词汇表。

In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [12]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 以下是该词汇表中的前 50 个条目：

In [13]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 下面，我们使用一个小词汇表来展示一个简短示例文本的分词过程：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="700px">

- 现在将其全部整合到一个分词器类中

In [19]:
import re  # 引入正则表达式模块，用于字符串处理

class SimpleTokenizerV1:
    # 初始化函数，接收一个词汇表（vocab）作为输入
    def __init__(self, vocab):
        # 将词汇表保存为一个从字符串到整数的字典（映射）
        self.str_to_int = vocab
        # 创建一个从整数到字符串的反向映射字典
        self.int_to_str = {i: s for s, i in vocab.items()}
    
    # 编码函数：将输入的文本转换为对应的整数 ID 序列
    def encode(self, text):
        # 使用正则表达式对文本进行分词
        # 分隔符包括标点符号（如 , . : ; ? _ ! " ' 等）、双连字符（--）以及空白字符（\s）
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        
        # 移除分词结果中的空字符串，并去掉每个分词项的首尾空格
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        
        # 将分词结果转换为对应的整数 ID 序列
        # 如果词汇表中没有找到某个分词项，会抛出 KeyError 异常
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    # 解码函数：将整数 ID 序列转换回原始文本
    def decode(self, ids):
        # 根据整数 ID 序列查找对应的字符串，并用空格拼接成文本
        text = " ".join([self.int_to_str[i] for i in ids])
        
        # 使用正则表达式修复文本中的空格问题
        # 将标点符号（如 , . ? ! " ' 等）前面的多余空格移除
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


- `encode` 函数将文本转换为标记 ID  
- `decode` 函数将标记 ID 转换回文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="800px">

- 我们可以使用分词器将文本编码（即分词）为整数  
- 这些整数随后可以作为嵌入（稍后）以用作 LLM 的输入

In [15]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


此时如果我们输入的词里面有 vocab 里面没有记录的，它会报错

In [16]:
input_text = "What if you don't know Bowei?"
input_token = tokenizer.encode(input_text)
print(input_token)

KeyError: 'Bowei'

- 我们可以将整数解码回文本

In [17]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [18]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊上下文标记

- 添加一些用于未知单词和表示文本结尾的“特殊”标记很有用

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="800px">

- 添加一些“特殊”标记对于表示未知单词和文本结尾很有帮助  
- 一些分词器使用特殊标记为 LLM 提供额外的上下文信息  
- 这些特殊标记包括：  
  - `[BOS]`（序列开头）标记文本的起始位置  
  - `[EOS]`（序列结尾）标记文本的结束位置（通常用于连接多个不相关的文本，例如两篇不同的维基百科文章或两本不同的书等）  
  - `[PAD]`（填充）当我们以大于 1 的批量大小训练 LLM 时（可能会包含长度不同的多个文本；通过填充标记将较短的文本填充到与最长文本相同的长度，从而使所有文本长度一致）  
- `[UNK]` 表示词汇表中未包含的单词  

- 需要注意的是，GPT-2 不需要上述提到的任何标记，而仅使用一个 `"<|endoftext|>"` 标记来降低复杂性  
- `"<|endoftext|>"` 类似于上述提到的 `[EOS]` 标记  
- GPT 也使用 `"<|endoftext|>"` 进行填充（因为在对批量输入进行训练时，我们通常会使用掩码，不会关注填充的标记，因此这些标记是什么并不重要）  
- GPT-2 不使用 `<UNK>` 标记来表示词汇表外的单词；相反，GPT-2 使用字节对编码（BPE）分词器，它将单词分解为子词单元，我们将在后面的部分讨论这一点。

- 我们在两个独立的文本来源之间使用 `"<|endoftext|>"` 标记：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="800px">

- 让我们看看如果对以下文本进行分词会发生什么：

In [16]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 上面的代码会产生错误，因为单词 "Hello" 不在词汇表中。  
- 为了处理这种情况，我们可以在词汇表中添加像 `"<|unk|>"` 这样的特殊标记，以表示未知单词。  
- 既然我们已经在扩展词汇表，那么让我们再添加另一个标记 `"<|endoftext|>"`，它在 GPT-2 训练中用于表示文本的结尾（同时也用于连接的文本之间，例如，如果我们的训练数据集包含多篇文章、书籍等）。

In [21]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [22]:
len(vocab.items())

1132

In [23]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 我们还需要相应地调整分词器，以便它知道何时以及如何使用新的 `<unk>` 标记。

In [25]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

Let's try to tokenize text with the modified tokenizer:

In [26]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [27]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [28]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 字节对编码 (BytePair Encoding)

- GPT-2 使用字节对编码 (BPE) 作为其分词器  
- 它允许模型将不在其预定义词汇表中的单词分解为更小的子词单元，甚至是单个字符，从而能够处理词汇表外的单词  
- 例如，如果 GPT-2 的词汇表中没有单词 "unfamiliarword"，它可能会将其分词为 ["unfam", "iliar", "word"] 或其他子词分解形式，具体取决于其训练的 BPE 合并规则  
- 原始的 BPE 分词器可以在这里找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)  
- 在本章中，我们使用的是 OpenAI 开源的 [tiktoken](https://github.com/openai/tiktoken) 库中的 BPE 分词器，该库使用 Rust 实现其核心算法以提高计算性能  
- 我在 [./bytepair_encoder](../02_bonus_bytepair-encoder) 中创建了一个笔记本，将这两种实现进行了并排比较（在样本文本上，tiktoken 的速度大约快了 5 倍）

In [29]:
#!pip install tiktoken

In [30]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [31]:
tokenizer = tiktoken.get_encoding("gpt2")

In [34]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [35]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE 分词器将未知单词分解为子词和单个字符：

In [36]:
#TODO: 练习把 Akwirw ier 输入 tokenize 并且逆向操作
interest_word = "Akwirw ier"
integer_token = tokenizer.encode(interest_word)
print(integer_token)
returned_word = tokenizer.decode(integer_token)
print(returned_word)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="600px">

## 2.6 使用滑动窗口进行数据采样

- 我们训练 LLM 一次生成一个单词，因此我们需要相应地准备训练数据，其中序列中的下一个单词代表要预测的目标：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="800px">

In [37]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 对于每个文本块，我们需要输入和目标  
- 由于我们希望模型预测下一个单词，目标是输入向右移动一个位置的结果

In [38]:
enc_sample = enc_text[50:]

In [39]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- One by one, the prediction would look like as follows:

In [40]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [41]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 我们将在后续章节中介绍注意力机制之后，再处理下一个词的预测问题  
- 目前，我们实现一个简单的数据加载器，它会遍历输入数据集并返回向右移动一位的输入和目标

- 安装并导入 PyTorch（有关安装提示，请参见附录 A）

In [42]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0


- 我们使用滑动窗口方法，每次将位置向右移动 +1：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="800px">

- 创建从输入文本数据集中提取块的数据集和数据加载器

In [44]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    """
    GPTDatasetV1 类是一个自定义的 PyTorch Dataset，用于准备语言模型训练数据。
    它将文本转换为重叠的序列对，每对包含输入序列和目标序列（输入序列向右偏移一个token）。
    这种设计使模型能够学习预测序列中的下一个token，这是语言模型预训练的标准方法。
    """
    def __init__(self, txt, tokenizer, max_length, stride):
        """
        初始化 GPTDatasetV1 对象。
        
        参数:
            txt (str): 要处理的原始文本
            tokenizer: 用于将文本转换为token ID的分词器
            max_length (int): 每个输入序列的最大长度
            stride (int): 滑动窗口的步长，决定了相邻序列之间的重叠程度
        """
        self.input_ids = []  # 存储所有输入序列
        self.target_ids = []  # 存储所有目标序列（每个都是对应输入序列向右偏移一位）

        # 使用tokenizer将整个文本编码为token ID序列
        # allowed_special参数允许特殊token "<|endoftext|>" 在文本中保留
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # 使用滑动窗口将token序列分割成重叠的块
        # 每次移动stride个位置，创建一个新的长度为max_length的序列
        for i in range(0, len(token_ids) - max_length, stride):
            # 提取当前位置的输入块
            input_chunk = token_ids[i:i + max_length]
            # 提取目标块（比输入块向右偏移一个位置）
            target_chunk = token_ids[i + 1: i + max_length + 1]
            # 将输入和目标块转换为PyTorch张量并存储
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        """
        返回数据集中的样本数量。
        
        返回:
            int: 数据集中的样本数量
        """
        return len(self.input_ids)

    def __getitem__(self, idx):
        """
        获取指定索引的样本。

        参数:
            idx (int): 要获取的样本的索引
            
        返回:
            tuple: 包含(输入序列, 目标序列)的元组
        """
        return self.input_ids[idx], self.target_ids[idx]


In [45]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    """
    创建一个用于GPT模型训练的PyTorch DataLoader。
    
    这个函数封装了数据集创建和数据加载器配置的过程，提供了一个简便的接口
    来准备训练数据。它使用GPTDatasetV1类处理文本数据，并配置DataLoader
    以适合语言模型训练的批处理方式。
    
    参数:
        txt (str): 要处理的原始文本
        batch_size (int, 可选): 每个批次包含的样本数，默认为4
        max_length (int, 可选): 每个序列的最大token数，默认为256
        stride (int, 可选): 滑动窗口的步长，决定序列间的重叠程度，默认为128
        shuffle (bool, 可选): 是否在每个epoch随机打乱数据，默认为True
        drop_last (bool, 可选): 如果最后一个批次不完整是否丢弃，默认为True
        num_workers (int, 可选): 用于数据加载的子进程数量，默认为0（在主进程中加载）
        
    返回:
        DataLoader: 配置好的PyTorch数据加载器，可直接用于模型训练
    """

    # 初始化tokenizer，使用GPT-2的编码方式
    # tiktoken是OpenAI开发的高效tokenizer库
    tokenizer = tiktoken.get_encoding("gpt2")

    # 创建数据集实例
    # 将原始文本转换为模型可用的输入-目标对序列
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 创建并配置DataLoader
    # - batch_size: 控制每批处理的样本数
    # - shuffle: 随机打乱数据，增强训练的随机性，防止模型记忆数据顺序
    # - drop_last: 如果为True，丢弃不足一个批次的最后几个样本
    # - num_workers: 控制数据加载的并行程度，0表示在主进程中加载
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader


- 我们来测试一下使用批量大小为1、上下文大小为4的数据加载器，看看它如何处理文本数据

In [46]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [47]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [48]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 以下是一个使用步幅等于上下文长度（此处为 4）的示例：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="800px">

- 我们还可以创建批处理输出  
- 注意，这里我们增加了步幅，以确保批次之间没有重叠，因为更多的重叠可能会导致过拟合加剧

In [49]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建标记嵌入 (Token Embeddings)

- 数据已经几乎准备好用于 LLM  
- 但最后，让我们通过嵌入层将标记转换为连续的向量表示  
- 通常，这些嵌入层是 LLM 的一部分，并在模型训练期间进行更新（训练）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="700px">

- 假设我们有以下四个输入示例，其输入 ID 分别为 2、3、5 和 1（分词后）：

In [50]:
input_ids = torch.tensor([2, 3, 5, 1])

- 为了简化起见，假设我们有一个仅包含 6 个单词的小词汇表，并且我们希望创建大小为 3 的嵌入：

In [51]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 这将生成一个 6x3 的权重矩阵：

In [55]:
print(embedding_layer.weight.shape)
print(embedding_layer.weight)

torch.Size([6, 3])
Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉独热编码 (one-hot encoding) 的人来说，上述嵌入层方法本质上只是在全连接层中实现独热编码后进行矩阵乘法的一种更高效的方式，相关内容在补充代码 [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul) 中有所描述。  
- 因为嵌入层只是独热编码和矩阵乘法方法的一种更高效的等效实现，所以它可以被视为一种可以通过反向传播进行优化的神经网络层。

- 要将 ID 为 3 的标记转换为三维向量，我们需要执行以下操作：

In [56]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 注意，上面的结果是 `embedding_layer` 权重矩阵的第 4 行  
- 要嵌入上述所有四个 `input_ids` 值，我们执行以下操作：

In [57]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一个查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="700px">

- **You may be interested in the bonus content comparing embedding layers with regular linear layers: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 编码单词位置

- 嵌入层将 ID 转换为相同的向量表示，而不考虑它们在输入序列中的位置：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="700px">

- 位置嵌入与标记嵌入向量结合，形成大型语言模型的输入嵌入：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="700px">

- BytePair 编码器的词汇表大小为 50,257：  
- 假设我们希望将输入标记编码为 256 维的向量表示：

In [58]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果我们从数据加载器中采样数据，我们会将每个批次中的标记嵌入到 256 维的向量中。  
- 如果我们的批量大小为 8，每个批量包含 4 个标记，这将生成一个 8 x 4 x 256 的张量：

In [59]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [62]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [63]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embedding)

torch.Size([8, 4, 256])


- GPT-2 使用绝对位置嵌入，因此我们只需创建另一个嵌入层：

In [66]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
print(pos_embedding_layer.weight)

Parameter containing:
tensor([[-0.8194,  0.5543, -0.8290,  ...,  0.1325,  0.2115,  0.3610],
        [ 0.4193, -0.9461, -0.3407,  ...,  0.7930,  1.7009,  0.5663],
        [-0.2362, -1.7187, -1.0489,  ...,  1.1218,  0.2796,  0.9912],
        [-0.9549,  0.4699,  0.2580,  ..., -1.3689,  1.6505,  1.3488]],
       requires_grad=True)


In [67]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])


- 要创建用于 LLM 的输入嵌入，我们只需将标记嵌入和位置嵌入相加：

In [68]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[-0.3281,  1.6782,  0.6298,  ..., -0.2670, -1.6620,  0.2165],
         [ 0.8674, -0.6925, -0.6063,  ...,  1.2927,  0.5018, -0.6181],
         [-0.4869, -1.7733, -0.3802,  ...,  2.0836,  2.6533,  0.9384],
         [-0.0091,  1.3356,  1.8771,  ..., -1.8233,  0.9045,  1.6972]],

        [[ 0.7267,  2.2912, -1.6138,  ...,  0.0321,  1.0699,  0.0189],
         [-1.4429, -1.1375, -0.7219,  ...,  1.9150,  1.3513,  1.1754],
         [ 1.7486, -2.3669, -1.1904,  ...,  0.7377, -0.6559,  2.4390],
         [ 0.0099,  1.7672, -1.3627,  ..., -0.2226,  3.2302,  1.7457]],

        [[-1.5907,  1.2115, -0.6627,  ..., -0.6719,  0.2657,  1.1036],
         [ 1.2239, -0.4414,  0.9515,  ...,  2.2578,  2.1106,  0.8868],
         [-0.1567, -3.4823, -0.4740,  ...,  3.3041,  2.1027,  0.6277],
         [-0.5282,  0.4051,  0.8265,  ..., -1.8898,  2.9570,  2.1961]],

        ...,

        [[-2.4349,  1.5153, -3.4727,  ..., -0.8320,  1.3004,  1.9994],
         [ 0.0208, -1.8696, -1.65

- 在输入处理流程的初始阶段，输入文本被分割为单独的标记。  
- 在此分割之后，这些标记会根据预定义的词汇表转换为标记 ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="700px">

# 总结与要点

请参阅 [./dataloader.ipynb](./dataloader.ipynb) 代码笔记本，这是本章中实现的数据加载器的简化版本，我们将在后续章节中用它来训练 GPT 模型。

练习题的解答请参阅 [./exercise-solutions.ipynb](./exercise-solutions.ipynb)。